Note:  This tutorial requires that the Wallaroo instance has 

In [ ]:
import uuid
import json
import os
import pandas

import wallaroo
from wallaroo.pipeline import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
import pyarrow as pa

if os.environ.get("WALLAROO_SDK_CREDENTIALS"):
    auth_type="user_password"
else:
    auth_type="none"

In [ ]:
wl = wallaroo.Client(auth_type=auth_type, interactive=True, request_timeout=90)

In [ ]:
input_schema = pa.schema([
    pa.field('inputs', pa.list_(
        pa.list_(
            pa.list_(
                pa.int64(),
                list_size=224
            ),
            list_size=224
        ),
        list_size=3
    )),
])

output_schema = pa.schema([
    pa.field('prediction', pa.list_(pa.float64(), list_size=1000)),
])

model = wl.register_model_image(
    name="alexnet-cuda",
    image=f"ghcr.io/wallaroolabs/wallaroo_tutorials/alexnet-gpu:1.30"
).configure("mlflow", input_schema=input_schema, output_schema=output_schema)

In [ ]:
pipeline_name = f"alexnet-cuda-pipeline"
pipeline = wl.build_pipeline(pipeline_name)
pipeline.add_model_step(model)

deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi').gpus(0) \
    .sidekick_gpus(model, 1) \
    .sidekick_env(model, {"GUNICORN_CMD_ARGS": "--timeout=180 --workers=1"}) \
    .build()
deployment_config

In [ ]:
pipeline.deploy(deployment_config=deployment_config)
pipeline.status()

In [ ]:
assert pipeline.status()['status'] == 'Running'

In [ ]:
pipeline.status()

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
input_data = {
        "inputs": [np.random.randint(0, 256, (3, 224, 224), dtype=np.uint8)] * 2, # required
}
dataframe = pd.DataFrame(input_data)
dataframe

In [ ]:
pipeline.infer_from_file('./data/test_alexnet_gpu.json')

In [ ]:
pipeline.undeploy()

In [ ]:
wl.list_pipelines()